In [28]:
import pandas as pd
import matplotlib.pyplot as plt
pd.options.display.float_format = '{:.3f}'.format
import numpy as np
import yfinance as yf
import pytz
import datetime
from os import system, name

#global Variables

stock, date = '', ''

Stock = yf.Ticker('TSLA')
options = Stock.option_chain('2020-07-23')
callT = options.calls
print(callT[callT['strike']  >= 1500])

          contractSymbol       lastTradeDate   strike  lastPrice     bid  \
171  TSLA200724C01500000 2020-07-17 19:59:49 1500.000    112.850 111.250   
172  TSLA200724C01505000 2020-07-17 19:59:56 1505.000    110.970 117.400   
173  TSLA200724C01510000 2020-07-17 19:59:53 1510.000    108.050 106.700   
174  TSLA200724C01515000 2020-07-17 19:58:53 1515.000    106.070 111.500   
175  TSLA200724C01520000 2020-07-17 19:59:09 1520.000    104.000 102.550   
..                   ...                 ...      ...        ...     ...   
249  TSLA200724C03100000 2020-07-17 19:58:25 3100.000      1.440   1.350   
250  TSLA200724C03200000 2020-07-17 19:58:08 3200.000      1.400   1.130   
251  TSLA200724C03300000 2020-07-17 19:50:15 3300.000      1.050   1.140   
252  TSLA200724C03400000 2020-07-17 19:59:57 3400.000      1.050   1.050   
253  TSLA200724C03500000 2020-07-17 19:59:58 3500.000      0.930   0.900   

        ask  change  percentChange   volume  openInterest  impliedVolatility  \
171 114

In [29]:
# getStock(st: String, d: String in date format) => calls: dataframe, puts: dataframe
def getStock(st, d):
    # global variables
    global stock
    global date
    
    # local variables
    stock = st
    date = d
    
    # assigning stock value and getting options data
    s = yf.Ticker(stock)
    opt = s.option_chain(date)
    call, put = opt.calls, opt.puts
    
    # Making our own Fair price column 'It is not exactly the market price, but very close.'
    call['Fair Price'] = [i if i >= 0.01 else 0.01 for i in (call['ask']+ call['bid'])/2]
    put['Fair Price']  = [i if i >= 0.01 else 0.01 for i in (put['ask'] + put['bid'])/2]
    
    # Shortlisting data to a specific possible (rational) price range
    currentPrice = s.history(period = "max")['Close'].iloc[-1]
    oldPrice = s.history(period = "max")['Low'].iloc[-20]
    r = currentPrice - oldPrice
    priceRange = [int((currentPrice - r).round()), int((currentPrice + r).round())]
  
    # Assigning values to calls and puts
    calls = call[call['strike'] <= priceRange[1]]
    puts = put[put['strike'] >= priceRange[0]]
    
    return calls, puts

calls = getStock('TSLA', '2020-07-23')[0]
puts = getStock('TSLA', '2020-07-23')[1]



In [30]:
# getData(prediction: dataframe (can be calls or puts)) => n: dataframe
def getData(prediction):
    
    # Shortlisting even more accoriding to good volume.
    c = prediction[prediction['volume']>=int(prediction['volume'].mean())]
    
    if c['ask'].mean() != 0: # need to use time library later on
        l = c[['strike', 'Fair Price']]
        m = l.set_index('strike').to_dict()
        n = m['Fair Price'] 
        print("Using Current Market Price")
        
    else:
        j = c[['strike', 'lastPrice']]
        m = j.set_index('strike').to_dict()
        n = m['lastPrice']
        print("Using Last Price")
        
    return n

In [31]:

def appendBest(best_ratio, l, s, lp, sp, r, mrx, mrw):
    
    best_ratio['Long(Buy)'] = l
    best_ratio['Short(Sell)'] = s
    best_ratio['Risk/Reward Ratio'] = r
    best_ratio['Short Premium'] = sp
    best_ratio['Long Premium'] = lp
    best_ratio['maxRisk'] = mrx
    best_ratio['maxReward'] = mrw

    
def rnR(type, sL, sS, pL, pS):
    
    # credit spreads
    if type == 'callC' or type == 'putC':
        MaxReward = pS - pL
        MaxRisk = (sL - sS - MaxReward)
        
        if type == 'putC':
            MaxRisk *= -1
    
    # debit spreads
    elif type == 'callD' or type == 'putD':    
        MaxRisk = pL - pS
        MaxReward = (sL - sS) - MaxRisk
        
        if type == 'callD':
            MaxReward *= -1
            
    return MaxRisk, MaxReward



In [32]:
#Spread 1
def basicSpreads(t):
    
    global calls, puts
    
    if t == 'callD' or t == 'callC':
        n = getData(calls)
    elif t == 'putC' or t == 'putD':
        n = getData(puts)
    
#     We need to check the risk, reward, break even point
#     Then we need to check the ratio of the risk/reward
#     Then we need to store the 'Transaction' of the least ratio

    # best_ratio: Dictionary to maintain the best available trade at a given time
    best_ratio = {'Short(Sell)':0, 'Long(Buy)':0, 'Risk/Reward Ratio':((2**31)-1), 'Short Premium':0, 'Long Premium':0, 'maxRisk':0, 'maxReward':0}
   
    # First loop to check what to short (sell)   
    for short in n:
        
        # Second loop to check what we're buying (long)
        for long in n:
            
            # In a credit call spread the long Strike cannot be lower or equal to the short strike price
            if t == 'putD' or t == 'callC':
                if long > short:

                    # Declaring variables to work with
                    strikeShort = short
                    strikeLong = long

                    premiumShort = n[short]
                    premiumLong = n[long]
                    
                    premiumShort = round(premiumShort, 2)
                    premiumLong = round(premiumLong, 2)
                    
                    z = rnR(t, strikeLong, strikeShort, premiumLong, premiumShort)
                
                    MaxRisk = z[0]
                    MaxReward = z[1]

                    if MaxReward > 0:
                        Ratio =  MaxRisk/MaxReward
                        if Ratio < best_ratio['Risk/Reward Ratio']:
                            appendBest(best_ratio, long, short, premiumLong, premiumShort, Ratio, MaxRisk, MaxReward)

               
            elif t == 'putC' or t == 'callD': 
                if long < short:

                    # Declaring variables to work with
                    strikeShort = short
                    strikeLong = long

                    premiumShort = n[short]
                    premiumLong = n[long]
                    
                    premiumShort = round(premiumShort, 2)
                    premiumLong = round(premiumLong, 2)

                    # Figuring out the max risk vs reward in each spread to minimise the risk/reward ratio.
                    z = rnR(t, strikeLong, strikeShort, premiumLong, premiumShort)
                
                    MaxRisk = z[0]
                    MaxReward = z[1]

                    if MaxReward > 0:
                        Ratio =  MaxRisk/MaxReward
                        if Ratio < best_ratio['Risk/Reward Ratio']:
                            appendBest(best_ratio, long, short, premiumLong, premiumShort, Ratio, MaxRisk, MaxReward)

#     plotGraph(best_ratio, 'Call Credit Spread', 'callC')
    return print("Best", t, "Spread: ", best_ratio)


In [33]:
def put_payoff(sT, k ,p):
    return np.where(sT<k, k-sT , 0)-p



def call_payoff(sT, k,p):
    return np.where(sT>k, sT-k , 0)-p



# def getBounds(sK, lK, t):
    
#     higher,lower = 0,0
    
#     if t=="putC" or t=='callD':
        
#         if currentPrice < sK:
#             lower = currentPrice-priceRange
            
#         else:
#             lower = sK-priceRange
            
#         if currentPrice>lK:
#             higher = currentPrice +priceRange
            
#         else:
#             higher = lK +priceRange
            
#     return upper,lower
   
    
def plotGraph(best_ratio, tittle, t):
    
    #long
    strike_price_long =best_ratio['Long(Buy)']
    premium_long= best_ratio['Long Premium']
    
    #Short
    strike_price_short = best_ratio['Short(Sell)']
    premium_short= best_ratio['Short Premium']
    
    #upper,lower = getBounds(strike_price_short,strike_price_long, t)
    #print(int(upper),int(lower))
    
    sT = np.arange(15,45,1)
    
    if t =='putC' or t =='putD':
        long_payoff = put_payoff(sT, strike_price_long,premium_long)
        short_payoff = put_payoff(sT, strike_price_short,premium_short)*-1
        
    if t == 'callC' or t =='callD':
        long_payoff = call_payoff(sT, strike_price_long,premium_long)
        short_payoff = call_payoff(sT, strike_price_short,premium_short)*-1
        
    final_payoff = short_payoff+long_payoff
    
    #plotting details
    fig, ax = plt.subplots()                       
    ax.plot(sT,long_payoff,'--', label = 'Long')
    plt.axhline(color='black')
    ax.plot(sT,short_payoff, '--', label = 'Short')
    ax.plot(sT,final_payoff, label = 'Final Payoff')
    plt.xlabel('Stock Price (sT)')
    plt.ylabel('Profit & Loss')
    ax.set_title(tittle)
    plt.legend()
    plt.show() 
    



In [34]:
basicSpreads('callD')
print("\n")
basicSpreads('putC')
print("\n")
basicSpreads('callC')
print("\n")
basicSpreads('putD')

Using Current Market Price
Best callD Spread:  {'Short(Sell)': 1515.0, 'Long(Buy)': 1510.0, 'Risk/Reward Ratio': -24.00000000000142, 'Short Premium': 112.65, 'Long Premium': 107.85, 'maxRisk': -4.800000000000011, 'maxReward': 0.19999999999998863}


Using Current Market Price
Best putC Spread:  {'Short(Sell)': 1565.0, 'Long(Buy)': 1560.0, 'Risk/Reward Ratio': 1.449640287769784, 'Short Premium': 157.05, 'Long Premium': 145.93, 'maxRisk': 16.120000000000005, 'maxReward': 11.120000000000005}


Using Current Market Price
Best callC Spread:  {'Short(Sell)': 1505.0, 'Long(Buy)': 1510.0, 'Risk/Reward Ratio': -0.5112414467253179, 'Short Premium': 118.08, 'Long Premium': 107.85, 'maxRisk': -5.230000000000004, 'maxReward': 10.230000000000004}


Using Current Market Price
Best putD Spread:  {'Short(Sell)': 1390.0, 'Long(Buy)': 1395.0, 'Risk/Reward Ratio': 0.03092783505154609, 'Short Premium': 62.25, 'Long Premium': 62.4, 'maxRisk': 0.14999999999999858, 'maxReward': 4.850000000000001}
